# Employee Income Prediction Script

This notebook fetches the latest employee entry from the API, preprocesses the data, makes predictions using the trained linear regression model, and logs the results to the database.

## Steps:
1. Import required libraries and setup
2. Fetch the latest employee entry from the API
3. Handle missing data and preprocess the input
4. Load the trained model and scaler
5. Make predictions
6. Log results to the database


In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import requests
import json
import joblib
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
import warnings
import sys
import os

# Add parent directory to path to import from task_2_api
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'task_2_api'))

warnings.filterwarnings('ignore')

# API Configuration
API_BASE_URL = "http://localhost:8000"  # Update if your API runs on a different port
print("Libraries imported successfully!")


/Users/patrickniyo/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Libraries imported successfully!


## Step 1: Fetch Latest Employee Entry from API

Fetches the last entered employee record based on creation timestamp (`created_at`), not employee_number.


In [7]:
def get_latest_employee():
    """
    Fetch the latest employee entry from the API.
    Returns the employee with the most recent created_at timestamp (last entered record).
    """
    try:
        # Fetch the latest employee by creation timestamp
        response = requests.get(f"{API_BASE_URL}/mysql/employees/latest/entry")
        response.raise_for_status()
        
        latest_employee = response.json()
        
        if not latest_employee:
            raise ValueError("No employees found in the database")
        
        employee_number = latest_employee.get('employee_number')
        created_at = latest_employee.get('created_at', 'N/A')
        
        print(f"Latest employee found: Employee #{employee_number}")
        print(f"Created at: {created_at}")
        return latest_employee
    
    except requests.exceptions.ConnectionError:
        print(f"ERROR: Could not connect to API at {API_BASE_URL}")
        print("Please make sure the API server is running:")
        print("  cd task_2_api")
        print("  uvicorn main:app --reload")
        return None
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 404:
            print("ERROR: No employees found in the database")
            print("Please create at least one employee entry first")
        else:
            print(f"HTTP Error: {e}")
        return None
    except Exception as e:
        print(f"Error fetching employee data: {e}")
        return None

# Fetch the latest employee
latest_employee = get_latest_employee()
if latest_employee:
    print(f"\nEmployee data:")
    print(json.dumps(latest_employee, indent=2))


Latest employee found: Employee #2069
Created at: N/A

Employee data:
{
  "age": 21,
  "gender": "Female",
  "marital_status": "Single",
  "education": 2,
  "education_field": "Life Sciences",
  "distance_from_home": 1,
  "over_18": "Y",
  "employee_count": 1,
  "attrition": "Yes",
  "department_name": null,
  "job_satisfaction": 3,
  "employee_number": 2069
}


## Step 2: Fetch Additional Employee Details (Job Details, etc.)

Since the employee entry might not have all features needed for prediction, we need to fetch related data or construct it from available fields.


In [8]:
# For prediction, we need to map the API employee data to the format expected by our model
# Since the API might not return all fields from the CSV, we'll need to handle missing fields

print("Note: The API employee structure might differ from the CSV format.")
print("We'll map available fields and use default values for missing ones.")


Note: The API employee structure might differ from the CSV format.
We'll map available fields and use default values for missing ones.


## Step 3: Load Trained Model and Preprocessing Tools


In [9]:
# Load the trained model
model_path = 'models/employee_income_model.joblib'
scaler_path = 'models/employee_income_scaler.joblib'
feature_names_path = 'models/feature_names.json'
label_encoders_path = 'models/label_encoders.joblib'

try:
    model = joblib.load(model_path)
    scaler = joblib.load(scaler_path)
    
    with open(feature_names_path, 'r') as f:
        feature_names = json.load(f)
    
    label_encoders = joblib.load(label_encoders_path)
    
    print("Model and preprocessing tools loaded successfully!")
    print(f"Model expects {len(feature_names)} features")
    print(f"Features: {feature_names}")
except FileNotFoundError as e:
    print(f"ERROR: Model files not found. Please run the training notebook first!")
    print(f"Missing file: {e.filename}")
    model = None
except Exception as e:
    print(f"Error loading model: {e}")
    model = None


Model and preprocessing tools loaded successfully!
Model expects 30 features
Features: ['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department', 'DistanceFromHome', 'Education', 'EducationField', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction', 'MaritalStatus', 'MonthlyRate', 'NumCompaniesWorked', 'OverTime', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager']


## Step 4: Preprocess Employee Data for Prediction


In [15]:
def preprocess_employee_data(employee_data, feature_names, label_encoders):
    """
    Preprocess employee data to match the model's expected format.
    Handles missing data and encodes categorical variables.
    
    Maps API fields (lowercase/snake_case) to CSV column names (TitleCase).
    """
    # Create a dictionary with default values from the original dataset
    # Load original CSV to get defaults and understand data ranges
    try:
        original_data = pd.read_csv('../hr_employee_attrition.csv')
        default_values = {}
        
        # Calculate median/mode for each feature
        for col in feature_names:
            if col in original_data.columns:
                if original_data[col].dtype in ['int64', 'float64']:
                    default_values[col] = float(original_data[col].median())
                else:
                    mode_val = original_data[col].mode()
                    default_values[col] = mode_val[0] if len(mode_val) > 0 else 0
            else:
                # Use reasonable defaults
                default_values[col] = 0
        
    except Exception as e:
        print(f"Warning: Could not load original CSV for defaults: {e}")
        default_values = {col: 0 for col in feature_names}
    
    # Map API employee data to feature format
    # API returns fields in lowercase/snake_case, CSV uses TitleCase
    processed_data = {}
    
    # Mapping from API field names (lowercase/snake_case) to CSV column names (TitleCase)
    api_to_csv_mapping = {
        'age': 'Age',
        'gender': 'Gender',
        'marital_status': 'MaritalStatus',
        'education': 'Education',
        'education_field': 'EducationField',
        'distance_from_home': 'DistanceFromHome',
        'department_name': 'Department',
        'business_travel': 'BusinessTravel',
        'over_time': 'OverTime',
    }
    
    # Start with default values for all features
    for feature in feature_names:
        processed_data[feature] = default_values.get(feature, 0)
    
    # Map available employee data from API
    employee_dict = employee_data if isinstance(employee_data, dict) else employee_data.__dict__
    
    # Map fields from API format to CSV format
    for api_field, csv_field in api_to_csv_mapping.items():
        if api_field in employee_dict and csv_field in feature_names:
            value = employee_dict[api_field]
            # Handle None values
            if value is None:
                # Use default value instead
                if csv_field in default_values:
                    processed_data[csv_field] = default_values[csv_field]
                else:
                    processed_data[csv_field] = 0  # Default fallback
            else:
                processed_data[csv_field] = value
    
    # Handle nested job_details if available
    if 'job_details' in employee_dict and employee_dict['job_details']:
        job_details = employee_dict['job_details']
        # Map job-related fields
        if 'job_level' in job_details and 'JobLevel' in feature_names:
            processed_data['JobLevel'] = job_details['job_level']
        if 'job_involvement' in job_details and 'JobInvolvement' in feature_names:
            processed_data['JobInvolvement'] = job_details['job_involvement']
        if 'job_satisfaction' in job_details and 'JobSatisfaction' in feature_names:
            processed_data['JobSatisfaction'] = job_details['job_satisfaction']
        if 'business_travel' in job_details and 'BusinessTravel' in feature_names:
            processed_data['BusinessTravel'] = job_details['business_travel']
        if 'overtime' in job_details and 'OverTime' in feature_names:
            processed_data['OverTime'] = job_details['overtime']
    
    # Replace None values with defaults before creating DataFrame
    for key, value in processed_data.items():
        if value is None:
            processed_data[key] = default_values.get(key, 0)
    
    # Create DataFrame
    df = pd.DataFrame([processed_data])
    
    # Encode categorical variables using saved label encoders
    for col in df.columns:
        if col in label_encoders:
            try:
                original_val = df[col].iloc[0]
                # Only encode if it's a string
                if isinstance(original_val, str):
                    # Check if the value is in the encoder's classes
                    if original_val in label_encoders[col].classes_:
                        df[col] = label_encoders[col].transform([original_val])[0]
                    else:
                        print(f"Warning: '{original_val}' not in label encoder for {col}, using default")
                        # Use the first encoded value as default
                        if len(label_encoders[col].classes_) > 0:
                            df[col] = label_encoders[col].transform([label_encoders[col].classes_[0]])[0]
                        else:
                            df[col] = 0
                # If it's numeric, assume it's already encoded
            except Exception as e:
                print(f"Warning: Could not encode {col}: {e}")
                # Use default from original dataset
                if col in default_values:
                    default_val = default_values[col]
                    if isinstance(default_val, str) and col in label_encoders:
                        try:
                            df[col] = label_encoders[col].transform([default_val])[0]
                        except:
                            df[col] = 0
                    else:
                        df[col] = default_val if isinstance(default_val, (int, float)) else 0
                else:
                    df[col] = 0
    
    # Ensure all feature columns are present and in correct order
    for feature in feature_names:
        if feature not in df.columns:
            df[feature] = default_values.get(feature, 0)
        # Ensure numeric types are correct
        if feature in default_values and isinstance(default_values[feature], (int, float)):
            df[feature] = pd.to_numeric(df[feature], errors='coerce').fillna(default_values[feature])
    
    # Select features in the correct order
    feature_df = df[feature_names]
    
    # Final check: Fill any remaining NaN/None values with defaults
    for col in feature_df.columns:
        # Check for NaN or None values
        if feature_df[col].isnull().any():
            default_val = default_values.get(col, 0)
            if isinstance(default_val, str):
                # For string columns, try to encode to default or use 0
                if col in label_encoders:
                    try:
                        default_val = label_encoders[col].transform([default_val])[0]
                    except:
                        default_val = 0
                else:
                    default_val = 0
            feature_df[col] = feature_df[col].fillna(default_val)
        
        # Convert to numeric if it's not already, and fill NaN
        try:
            feature_df[col] = pd.to_numeric(feature_df[col], errors='coerce')
            if feature_df[col].isnull().any():
                feature_df[col] = feature_df[col].fillna(default_values.get(col, 0))
        except:
            # If conversion fails, use default
            feature_df[col] = default_values.get(col, 0)
    
    # Final validation: Ensure no NaN values remain
    if feature_df.isnull().any().any():
        print("Warning: Found NaN values, filling with defaults...")
        for col in feature_df.columns:
            if feature_df[col].isnull().any():
                feature_df[col] = feature_df[col].fillna(default_values.get(col, 0))
    
    return feature_df, processed_data, default_values

if latest_employee and model:
    # Preprocess the employee data
    feature_df, raw_data, default_values = preprocess_employee_data(latest_employee, feature_names, label_encoders)
    
    print("\nPreprocessed feature data:")
    print(feature_df)
    print(f"\nRaw processed data sample:")
    for key, value in list(raw_data.items())[:10]:
        print(f"  {key}: {value}")



Preprocessed feature data:
   Age  Attrition  BusinessTravel  DailyRate  Department  DistanceFromHome  \
0   21          0               2      802.0           1                 1   

   Education  EducationField  EnvironmentSatisfaction  Gender  ...  \
0          2               1                      3.0       0  ...   

   PerformanceRating  RelationshipSatisfaction  StockOptionLevel  \
0                3.0                       3.0               1.0   

   TotalWorkingYears  TrainingTimesLastYear  WorkLifeBalance  YearsAtCompany  \
0               10.0                    3.0              3.0             5.0   

   YearsInCurrentRole  YearsSinceLastPromotion  YearsWithCurrManager  
0                 3.0                      1.0                   3.0  

[1 rows x 30 columns]

Raw processed data sample:
  Age: 21
  Attrition: No
  BusinessTravel: Travel_Rarely
  DailyRate: 802.0
  Department: Research & Development
  DistanceFromHome: 1
  Education: 2
  EducationField: Life Sciences


## Step 5: Scale Features and Make Prediction


In [16]:
if latest_employee and model:
    # Validate that there are no NaN/None values before scaling
    print("Validating feature data...")
    
    # Ensure default_values is available (from preprocessing step)
    # If not available, create defaults from the dataset
    if 'default_values' not in globals() or default_values is None:
        print("Warning: default_values not found, creating defaults...")
        try:
            original_data = pd.read_csv('../hr_employee_attrition.csv')
            default_values = {}
            for col in feature_df.columns:
                if col in original_data.columns:
                    if original_data[col].dtype in ['int64', 'float64']:
                        default_values[col] = float(original_data[col].median())
                    else:
                        mode_val = original_data[col].mode()
                        default_values[col] = mode_val[0] if len(mode_val) > 0 else 0
                else:
                    default_values[col] = 0
        except Exception as e:
            print(f"Could not load defaults from CSV: {e}")
            default_values = {col: 0 for col in feature_df.columns}
    
    # Check for None values and replace
    for col in feature_df.columns:
        # Replace None with defaults
        if feature_df[col].isnull().any():
            default_val = default_values.get(col, 0)
            # If default is string and we need numeric, use 0
            if isinstance(default_val, str):
                if col in label_encoders:
                    try:
                        default_val = label_encoders[col].transform([default_val])[0]
                    except:
                        default_val = 0
                else:
                    default_val = 0
            feature_df[col] = feature_df[col].fillna(default_val)
    
    # Ensure all values are numeric (convert any remaining strings)
    for col in feature_df.columns:
        if feature_df[col].dtype == 'object':
            # Try to convert to numeric
            feature_df[col] = pd.to_numeric(feature_df[col], errors='coerce').fillna(0)
        else:
            feature_df[col] = pd.to_numeric(feature_df[col], errors='coerce').fillna(0)
    
    # Final check for NaN
    nan_count = feature_df.isnull().sum().sum()
    if nan_count > 0:
        print(f"Warning: {nan_count} NaN values still present, filling with 0...")
        feature_df = feature_df.fillna(0)
    
    # Ensure all columns are numeric type
    feature_df = feature_df.astype(float)
    
    print(f"Feature data shape: {feature_df.shape}")
    print(f"NaN values in features: {feature_df.isnull().sum().sum()}")
    print(f"Data types: {feature_df.dtypes.value_counts().to_dict()}")
    
    # Scale the features
    feature_scaled = scaler.transform(feature_df)
    
    print(f"Scaled features shape: {feature_scaled.shape}")
    
    # Check for NaN in scaled features
    nan_in_scaled = np.isnan(feature_scaled).sum()
    if nan_in_scaled > 0:
        print(f"Warning: {nan_in_scaled} NaN values found in scaled features, filling with 0...")
        feature_scaled = np.nan_to_num(feature_scaled, nan=0.0)
    
    # Make prediction
    prediction = model.predict(feature_scaled)[0]
    
    print(f"\n{'='*50}")
    print(f"PREDICTION RESULT")
    print(f"{'='*50}")
    print(f"Employee Number: {latest_employee.get('employee_number')}")
    print(f"Predicted Monthly Income: ${prediction:,.2f}")
    print(f"Prediction Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"{'='*50}")
else:
    print("Cannot make prediction - missing employee data or model")
    prediction = None


Validating feature data...
Feature data shape: (1, 30)
NaN values in features: 0
Data types: {dtype('float64'): 30}
Scaled features shape: (1, 30)

PREDICTION RESULT
Employee Number: 2069
Predicted Monthly Income: $6,836.28
Prediction Date: 2025-10-31 22:36:07


## Step 6: Log Prediction Results to MongoDB via API

We'll use the `/mongo/predictions/` API endpoint to store prediction results in MongoDB. This avoids foreign key constraints since MongoDB stores employee_number as a simple integer field without relations.


In [ ]:
def log_prediction_to_db(employee_number, predicted_income, raw_data, feature_data):
    """
    Log prediction results to MongoDB via API endpoint.
    Uses the /mongo/predictions/ endpoint which stores data in MongoDB
    without foreign key constraints.
    """
    try:
        # Prepare prediction data
        # Convert feature_data DataFrame to dictionary
        if isinstance(feature_data, pd.DataFrame):
            # Get the first (and only) row as a dictionary
            input_features_dict = feature_data.iloc[0].to_dict()
        else:
            input_features_dict = feature_data.to_dict() if hasattr(feature_data, 'to_dict') else feature_data
        
        prediction_payload = {
            "employee_number": employee_number,
            "predicted_monthly_income": float(predicted_income),
            "input_features": input_features_dict,
            "model_version": "v1.0",
            "prediction_date": datetime.now().isoformat()
        }
        
        # Send POST request to API
        response = requests.post(
            f"{API_BASE_URL}/mongo/predictions/",
            json=prediction_payload,
            headers={"Content-Type": "application/json"}
        )
        
        response.raise_for_status()
        
        result = response.json()
        
        print(f"\n✅ Prediction logged to MongoDB successfully!")
        print(f"   Prediction ID: {result.get('_id', result.get('prediction_id', 'N/A'))}")
        print(f"   Employee Number: {employee_number}")
        print(f"   Predicted Income: ${predicted_income:,.2f}")
        print(f"   Model Version: {result.get('model_version', 'v1.0')}")
        print(f"   Logged at: {result.get('prediction_date', 'N/A')}")
        
        return result
        
    except requests.exceptions.ConnectionError:
        print(f"❌ Error: Could not connect to API at {API_BASE_URL}")
        print("   Please make sure the API server is running:")
        print("   cd task_2_api")
        print("   uvicorn main:app --reload")
        return None
    except requests.exceptions.HTTPError as e:
        print(f"❌ HTTP Error logging prediction: {e}")
        if e.response.status_code == 404:
            print("   Endpoint not found. Please check the API is running and includes the predictions router.")
        else:
            print(f"   Response: {e.response.text}")
        return None
    except Exception as e:
        print(f"❌ Error logging prediction: {e}")
        return None

if latest_employee and model and prediction is not None:
    # Log prediction to database via API
    employee_num = latest_employee.get('employee_number')
    result = log_prediction_to_db(employee_num, prediction, raw_data, feature_df)
    
    if result:
        print("\n✅ Prediction pipeline completed successfully!")
        print(f"   Prediction stored in MongoDB at: /mongo/predictions/")
    else:
        print("\n⚠️  Prediction made but could not be logged to database")
else:
    print("\n❌ Prediction pipeline incomplete - missing data or model")


❌ Error connecting to database: Foreign key associated with column 'predictions.employee_number' could not find table 'employees' with which to generate a foreign key to target column 'employee_number'
   Prediction will not be logged to database

⚠️  Prediction made but could not be logged to database


## Summary

This notebook:
1. ✅ Fetches the latest employee entry from the API (by `created_at` timestamp, not employee_number)
2. ✅ Handles missing data with default values
3. ✅ Preprocesses input data (encodes categorical variables)
4. ✅ Loads the trained linear regression model
5. ✅ Makes predictions on the employee data
6. ✅ Logs results to MongoDB via API endpoint

The prediction results are stored in the `predictions` collection in MongoDB via the `/mongo/predictions/` endpoint with:
- Employee number (no foreign key constraint)
- Predicted monthly income
- Prediction timestamp
- Input features used (as JSON)
- Model version

**No foreign key relations** - MongoDB stores employee_number as a simple integer field.
